In [1]:
import pandas as pd
import numpy as np

import quandl
import yfinance as yf
import pandas_datareader.data as web

In [2]:
file_key = open("../../../keys/quandl_key.txt","r")
API_KEY = file_key.read()
file_key.close()

quandl.ApiConfig.api_key = API_KEY

# Parameters

In [3]:
#RF_SOURCE = 'FRED'
RF_SOURCE = 'FRENCH'
#RF_SOURCE = 'ETF'

t0 = "1990-12-31"
t1 = "2022-04-30"

# Commodity Data
API required

In [4]:
info = pd.DataFrame(columns=['Asset Class','Name', 'Source', 'Ticker'])

info.loc['CL1'] = ['Future','NYMEX Crude Oil Futures','Quandl','CHRIS/CME_CL1']                    
info.loc['NG1'] = ['Future','NYMEX Natural Gas Futures','Quandl','CHRIS/CME_NG1']    
info.loc['KC1'] = ['Future','Coffee Futures','Quandl','CHRIS/ICE_KC1']
info.loc['CC1'] = ['Future','Cocoa Futures','Quandl','CHRIS/ICE_CC1']
info.loc['LB1'] = ['Future','Lumber Futures','Quandl','CHRIS/CME_LB1']                    
info.loc['CT1'] = ['Future','Cotton No. 2 Futures','Quandl','CHRIS/ICE_CT1']
info.loc['SB1'] = ['Future','Sugar No. 11 Futures','Quandl','CHRIS/ICE_SB1']
info.loc['LC1'] = ['Future','Cattle, CME Live Cattle Futures','Quandl','CHRIS/CME_LC1']    
info.loc['W1'] = ['Future','Wheat, CBOT Wheat Futures','Quandl','CHRIS/CME_W1']    
info.loc['S1'] = ['Future','Soybean, CBOT Soybean Futures','Quandl','CHRIS/CME_S1']    
info.loc['C1'] = ['Future','Corn, CBOT Corn Futures','Quandl','CHRIS/CME_C1']    
info.loc['GC1'] = ['Future','Gold, COMEX Gold Futures','Quandl','CHRIS/CME_GC1']    
info.loc['SI1'] = ['Future','Silver, COMEX Silver Futures','Quandl','CHRIS/CME_SI1']
info.loc['HG1'] = ['Future','Copper Futures','Quandl','CHRIS/CME_HG1']    
info.loc['PA1'] = ['Future','Palladium, NYMEX Palladium Futures','Quandl','CHRIS/CME_PA1']

info_futures = info[info['Asset Class'].str.contains('Future')]
tickers_futures = list(info_futures['Ticker'])
df = pd.DataFrame()
for tick in tickers_futures:
    df[tick] = quandl.get(tick, start_date=t0, end_date=t1)['Settle']

df.columns = list(info_futures.index)

In [5]:
prices = df.resample('M').last()
rets = prices.pct_change()
rets.dropna(inplace=True)

# Get Risk-free rate

Use short-term treasury bills.

Pros / cons of various sources.

* Get it from Ken French database via pandas-datareader, but updated with a few months lag.
* Could get it from FRED, but need API-KEY
* Could use ETF like SHV, but only goes back to 2007

In [6]:
if RF_SOURCE=='FRED':
    1+1

elif RF_SOURCE=='FRENCH':
    rawdata = web.DataReader('F-F_Research_Data_Factors', data_source='famafrench',start=t0,end=t1)
    ff = rawdata[0] / 100
    # convert the yyyy-mm to timestamp object. default is to put first of month, but should be end of month
    ff = ff.to_timestamp().resample('M').last()
    rf = ff[['RF']]
    
elif RF_SOURCE=='ETF':
    shv_raw = yf.download(['SHV'], start=t0, end=t1)['Adj Close']
    shv = shv_raw.resample('M').last()
    rf = shv.pct_change().iloc[1:].to_frame()
    rf.columns = ['RF']
    
info.loc['RF'] = RF_SOURCE

In [7]:
rets_all = rets.merge(rf,how='inner',on='Date')
rets = rets_all.drop(columns=['RF'])
retsx = rets_all.sub(rets_all['RF'],axis=0).drop(columns=['RF'])

In [8]:
with pd.ExcelWriter('commodities_return_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    rets.to_excel(writer, sheet_name='total returns')    
    retsx.to_excel(writer, sheet_name='excess returns')